# Caterpillar Tube Pricing
## Environment : Python 3
## Zhiqiang Zhong
## 05/12/2017

### import packages 

In [85]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline 

### read all 21 csvs

In [45]:
bill_of_materials_df = pd.read_csv('../competition_data/bill_of_materials.csv')
comp_adaptor_df = pd.read_csv('../competition_data/comp_adaptor.csv')
comp_boss_df = pd.read_csv('../competition_data/comp_boss.csv')
comp_elbow_df = pd.read_csv('../competition_data/comp_elbow.csv')
comp_float_df = pd.read_csv('../competition_data/comp_float.csv')
comp_hfl_df = pd.read_csv('../competition_data/comp_hfl.csv')
comp_nut_df = pd.read_csv('../competition_data/comp_nut.csv')
comp_other_df = pd.read_csv('../competition_data/comp_other.csv')
comp_sleeve_df = pd.read_csv('../competition_data/comp_sleeve.csv')
comp_straight_df = pd.read_csv('../competition_data/comp_straight.csv')
comp_tee_df = pd.read_csv('../competition_data/comp_tee.csv')
comp_threaded_df = pd.read_csv('../competition_data/comp_threaded.csv')
components_df = pd.read_csv('../competition_data/components.csv')
specs_df = pd.read_csv('../competition_data/specs.csv')
test_set_df = pd.read_csv('../competition_data/test_set.csv')
train_set_df = pd.read_csv('../competition_data/train_set.csv')
tube_end_form_df = pd.read_csv('../competition_data/tube_end_form.csv')
tube_df = pd.read_csv('../competition_data/tube.csv')
type_component_df = pd.read_csv('../competition_data/type_component.csv')
type_connection_df = pd.read_csv('../competition_data/type_connection.csv')
type_end_form_df = pd.read_csv('../competition_data/type_end_form.csv')

# data cleaning and preparation

In [46]:
train_set_df.quote_date = pd.to_datetime(train_set_df.quote_date)

In [47]:
test_set_df.quote_date = pd.to_datetime(test_set_df.quote_date)

In [87]:
#bill_of_materials_df

#how to replace data that has null id but numeric quantity???

In [69]:
components_df.component_id.replace("9999", "other", inplace=True)

In [86]:
tube_df['tube_assembly_id'] = pd.Series(tube_df['tube_assembly_id'], dtype = 'category')
tube_df['material_id'] = pd.Series(tube_df['material_id'], dtype = 'category')
tube_df['end_a_1x'] = pd.Series(tube_df['end_a_1x'], dtype = 'category')
tube_df['end_a_2x'] = pd.Series(tube_df['end_a_2x'], dtype = 'category')
tube_df['end_x_1x'] = pd.Series(tube_df['end_x_1x'], dtype = 'category')
tube_df['end_x_2x'] = pd.Series(tube_df['end_x_2x'], dtype = 'category')
tube_df['end_a'] = pd.Series(tube_df['end_a'], dtype = 'category')
tube_df['end_x'] = pd.Series(tube_df['end_x'], dtype = 'category')
# replace 9999.0 entries in bend_radius column with np.nan entries
tube_df = tube_df.replace(9999.0, np.nan)
tube_df = tube_df.replace('9999', 'other')
np.shape(tube_df)

## deleting rows with tube length of zero
#tube_df[tube_df['length'] == 0.0].index
#tube_df = tube_df.drop(tube_df[tube_df['length'] == 0.0].index)

(21198, 16)

In [89]:
tube_end_form_df.head()

,end_form_id,forming
0,EF-001,Yes
1,EF-002,No
2,EF-003,No
3,EF-004,No
4,EF-005,Yes


In [97]:
bill_of_materials_df.head()

,tube_assembly_id,component_id_1,quantity_1,component_id_2,quantity_2,component_id_3,quantity_3,component_id_4,quantity_4,component_id_5,quantity_5,component_id_6,quantity_6,component_id_7,quantity_7,component_id_8,quantity_8
0,TA-00001,C-1622,2.0,C-1629,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TA-00002,C-1312,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,TA-00003,C-1312,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,TA-00004,C-1312,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,TA-00005,C-1624,1.0,C-1631,1.0,C-1641,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [98]:
comp_adaptor_df.head()

,component_id,component_type_id,adaptor_angle,overall_length,end_form_id_1,connection_type_id_1,length_1,thread_size_1,thread_pitch_1,nominal_size_1,end_form_id_2,connection_type_id_2,length_2,thread_size_2,thread_pitch_2,nominal_size_2,hex_size,unique_feature,orientation,weight
0,C-0005,CP-028,NaN,58.4,A-001,B-001,NaN,1.312,12.0,NaN,A-001,B-004,NaN,1.000,11.5,NaN,34.93,No,No,0.206
1,C-0006,CP-028,NaN,34.8,A-001,B-001,NaN,0.437,20.0,NaN,A-001,B-005,NaN,0.750,16.0,NaN,22.20,No,No,0.083
2,C-1435,CP-028,NaN,20.3,A-007,B-004,NaN,NaN,NaN,15.88,A-001,B-007,NaN,0.875,18.0,NaN,22.22,No,No,0.023
3,C-1546,CP-028,NaN,26.4,A-007,B-004,NaN,0.125,27.0,NaN,A-001,B-004,NaN,0.125,27.0,NaN,15.88,No,No,0.026
4,C-1583,CP-028,NaN,44.5,A-001,B-005,NaN,1.312,12.0,NaN,A-007,B-005,NaN,1.062,12.0,NaN,38.10,No,No,0.256


In [99]:
components_df.head()

,component_id,name,component_type_id
0,other,OTHER,OTHER
1,C-0001,SLEEVE,CP-024
2,C-0002,SLEEVE,CP-024
3,C-0003,SLEEVE-FLARED,CP-024
4,C-0004,NUT,CP-026


In [118]:
#merge1: train + tube_df

merge1 = train_set_df.merge(tube_df)

In [119]:
#merge2: train + tube_df + bill_of_materials_df(bill_of_materials_summary_df)
bill_comp_types_df = bill_of_materials_df.iloc[:,[1,3,5,7,9,11,13,15]]
bill_comp_types_logical_df = ~bill_comp_types_df.isnull()
component_series = bill_comp_types_logical_df.sum(axis = 1)

bill_comp_quants_df = bill_of_materials_df.iloc[:,[2,4,6,8,10,12,14,16]]
quants_series = bill_comp_quants_df.sum(axis = 1)

bill_of_materials_summary_df = bill_of_materials_df.copy()
bill_of_materials_summary_df['type_totals'] = component_series
bill_of_materials_summary_df['component_totals'] = quants_series

merge2 = merge1.merge(bill_of_materials_summary_df[['tube_assembly_id', 'type_totals', 'component_totals']])

In [120]:
#merge3: train + tube_df + bill_of_materials_df(bill_of_materials_summary_df) + specs_df(totals_spec)
specS_only_df = specs_df.iloc[:, 1:11]
specs_logical_df = ~specs_only_df.isnull()
specs_totals = specs_logical_df.sum(axis=1)

specs_with_totals_df = specs_df.copy()
specs_with_totals_df['spec_totals'] = specs_totals

merge3 = merge2.merge(specs_with_totals_df[['tube_assembly_id', 'spec_totals']])

In [122]:
merge3.columns

Index(['tube_assembly_id', 'supplier', 'quote_date', 'annual_usage',
       'min_order_quantity', 'bracket_pricing', 'quantity', 'cost',
       'material_id', 'diameter', 'wall', 'length', 'num_bends', 'bend_radius',
       'end_a_1x', 'end_a_2x', 'end_x_1x', 'end_x_2x', 'end_a', 'end_x',
       'num_boss', 'num_bracket', 'other', 'type_totals', 'component_totals',
       'spec_totals'],
      dtype='object')